# Judge's Familiar – Introducción del Proyecto

**Judge's Familiar** es un asistente de IA diseñado para actuar como el compañero definitivo de reglas para jugadores de *Magic: The Gathering*. Más que un simple buscador, funciona como un **experto consultor en tiempo real**, capaz de interpretar dudas en lenguaje natural y explicar interacciones complejas basándose estrictamente en la documentación oficial.

El núcleo del sistema utiliza una arquitectura **RAG (Retrieval-Augmented Generation)** que combina:

* Las **Comprehensive Rules (CR)** oficiales de Magic (incluyendo el Glosario).
* El **Oracle text** de la base de datos de cartas (vía MTGJSON).

A diferencia de los LLMs genéricos que pueden "alucinar" reglas, *Judge's Familiar* recupera las normas exactas y construye una respuesta razonada, garantizando **trazabilidad y precisión**.

## Objetivo de la Versión 1 (V1)

La primera versión se define como un **"Pocket Companion"** (Compañero de Bolsillo) enfocado en la resolución de dudas técnicas.

* **Entrada Multimodal:**
    * Texto (consultas directas).
    * Voz (transcripción automática mediante modelo **Whisper**).
* **Salida Transparente:**
    * Explicación pedagógica de la interacción ("*Por qué* ocurre esto").
    * **Citas explícitas obligatorias** (ej. `[702.19b]`, `[510.1c]`).
    * Referencias cruzadas entre definiciones del Glosario y reglas numéricas.
* **Filosofía de Diseño:**
    * **Objetividad:** El sistema explica la mecánica, no juzga la conducta de los jugadores.
    * **Rigor:** Si la información no existe en las reglas recuperadas, el sistema lo indica en lugar de inventar.

*Judge's Familiar* no pretende reemplazar al juez humano en política de torneos o disputas de conducta; su misión es **democratizar el acceso a las reglas**, permitiendo partidas más fluidas y justas tanto en entornos casuales como competitivos.

## Arquitectura Técnica

El sistema se estructura en un pipeline de cuatro capas:

1.  **Ingesta de Datos (ETL)**
    * Parsing atómico de las *Comprehensive Rules* y el *Glosario* (1 regla = 1 nodo).
    * Indexación de cartas y textos Oracle.
2.  **Índices Semánticos**
    * Base de datos vectorial optimizada para búsquedas de similitud (embeddings).
3.  **Motor RAG + LLM**
    * Recuperación híbrida de reglas y definiciones.
    * Generación de respuesta con *Prompt Engineering* estricto (rol de Asistente Nivel 3).
    * Uso de modelos eficientes (`gpt-4o-mini`) con temperatura 0.0 para máxima fidelidad.
4.  **Interfaz de Usuario**
    * Chat web responsive (Desktop/Móvil).
    * Visualización clara de la respuesta separada de las fuentes técnicas.

## Escalabilidad (Roadmap)

La arquitectura modular permite futuras extensiones sin reescribir el núcleo:

* **Reconocimiento Visual:** Identificación de cartas físicas mediante cámara (OCR/Image Recognition).
* **Búsqueda Semántica por Arte:** Localización de cartas basada en descripciones visuales (*"bestia verde con tres cabezas"*).
* **Agentes de Decisión:** Una capa superior opcional para simular juicios complejos encadenados.

## Experiencia Interactiva

Como parte de la presentación, se entregarán copias físicas de la carta *Judge's Familiar* modificadas con un **código QR dinámico**. Esto permitirá a la audiencia y al jurado escanear la carta y **probar el sistema en vivo desde sus propios dispositivos**, cerrando la brecha entre el juego físico (Tabletop) y la asistencia digital.

In [ ]:
# -----------------------------------------------------------------------------
# JUDGE'S FAMILIAR - PRODUCTION V6
# -----------------------------------------------------------------------------

!pip install -q python-dotenv jedi llama_index llama-index-embeddings-openai llama-index-llms-openai

import os
import re
import json
import shutil
import random
import unicodedata
import requests
import time
from pathlib import Path
from typing import List, Dict, Any, Tuple
from tqdm import tqdm

from dotenv import load_dotenv

from llama_index.core import (
    Document,
    VectorStoreIndex,
    Settings,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage, MessageRole

# --- CONFIGURATION ---
LLM_MODEL_NAME = "gpt-4o-mini"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
FORCE_REBUILD = True  # Set True to load the new V5.8 Prompt

# --- 1. SETUP & AUTH ---
def load_openai_key():
    try:
        from google.colab import userdata
        key = userdata.get("OPENAI_API_KEY")
        if key:
            os.environ["OPENAI_API_KEY"] = key
            return
    except ImportError:
        pass
    load_dotenv()

load_openai_key()

# --- 2. PATHS & DIRECTORIES ---
BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "data"
DATA_DIR.mkdir(exist_ok=True)

RULES_STORAGE = DATA_DIR / "storage_rules"
CR_TXT_PATH = DATA_DIR / "magic_comprehensive_rules.txt"
CR_URL = "https://media.wizards.com/2025/downloads/MagicCompRules%2020251114.txt"

CARDS_STORAGE = DATA_DIR / "storage_cards"
ATOMIC_URL = "https://mtgjson.com/api/v5/AtomicCards.json"
ATOMIC_PATH = DATA_DIR / "AtomicCards.json"
CARDS_JSONL = DATA_DIR / "mtg_cards_clean.jsonl"

# --- 3. HELPER FUNCTIONS ---
def clean_unicode(s: str) -> str:
    s = unicodedata.normalize("NFC", s)
    return "".join(ch for ch in s if (unicodedata.category(ch) not in {"Cf", "Cc", "Cs", "Co", "Cn"} or ch in ("\n", "\t")))

def clean_val(val):
    return str(val).strip() if val is not None else ""

# ==============================================================================
# PART A: INGESTION LOGIC (Full Observability)
# ==============================================================================

def parse_rules_logic(txt_path: Path) -> List[Dict[str, Any]]:
    text = txt_path.read_text(encoding="utf-8")
    lines = text.splitlines()
    chapter_pattern = re.compile(r"^([1-9])\.\s+(.*)$")
    section_pattern = re.compile(r"^(\d{3})\.\s+(.*)$")
    rule_pattern = re.compile(r"^(\d{3}\.\d+[a-z]?)\.?\s+(.*)$")

    nodes = []
    current_chapter_id = "1"; current_chapter_title = "Game Concepts"
    current_section_id = "100"; current_section_title = "General"
    current_rule_id = None; current_rule_lines = []
    glossary_mode = False; glossary_lines = []; rules_parsed_count = 0

    print("[LOG] Starting Rules Parsing...")
    for line in lines:
        stripped = line.strip()
        if not stripped:
            if glossary_mode: glossary_lines.append(line)
            continue
        if stripped == "Glossary":
            if rules_parsed_count > 100:
                if current_rule_id:
                    nodes.append({"rule_id": current_rule_id, "chapter_id": current_chapter_id, "chapter_title": current_chapter_title, "section_id": current_section_id, "section_title": current_section_title, "text": "\n".join(current_rule_lines).strip()})
                    current_rule_id = None
                glossary_mode = True; continue
            else: continue
        if stripped == "Credits":
            if glossary_mode: break
            continue
        if not glossary_mode:
            m_chap = chapter_pattern.match(line)
            if m_chap: current_chapter_id = m_chap.group(1); current_chapter_title = m_chap.group(2).strip(); continue
            m_sec = section_pattern.match(line)
            if m_sec: current_section_id = m_sec.group(1); current_section_title = m_sec.group(2).strip(); continue
            m_rule = rule_pattern.match(line)
            if m_rule:
                if current_rule_id:
                    nodes.append({"rule_id": current_rule_id, "chapter_id": current_chapter_id, "chapter_title": current_chapter_title, "section_id": current_section_id, "section_title": current_section_title, "text": "\n".join(current_rule_lines).strip()})
                current_rule_id = m_rule.group(1); current_rule_lines = [m_rule.group(2).strip()]; rules_parsed_count += 1
                if not current_section_id: current_section_id = current_rule_id.split('.')[0]
                if not current_chapter_id: current_chapter_id = current_section_id[0]
            elif current_rule_id: current_rule_lines.append(line)
        else: glossary_lines.append(line)

    if glossary_lines:
        print(f"[LOG] Processing {len(glossary_lines)} lines of glossary...")
        full_text = "\n".join(glossary_lines)
        entries = re.split(r'\n\s*\n', full_text.strip())
        for entry in entries:
            parts = entry.strip().split('\n', 1)
            if not parts[0]: continue
            term = parts[0].strip()
            definition = parts[1].strip() if len(parts) > 1 else term
            nodes.append({"rule_id": term, "chapter_id": "G", "chapter_title": "Glossary", "section_id": None, "section_title": None, "text": definition})

    # --- STATISTICS ---
    rule_nodes = [r for r in nodes if r['chapter_id'] != 'G']
    glossary_nodes = [r for r in nodes if r['chapter_id'] == 'G']
    print("\n" + "="*50)
    print("DATASET STATISTICS (RULES)")
    print(f"   Total Nodes:      {len(nodes)}")
    print(f"   Numbered Rules:   {len(rule_nodes)}")
    print(f"   Glossary Terms:   {len(glossary_nodes)}")
    print("="*50 + "\n")
    return nodes

def process_cards_data():
    if not ATOMIC_PATH.exists():
        print(f"[LOG] Downloading AtomicCards.json...")
        with requests.get(ATOMIC_URL, stream=True) as r:
            with open(ATOMIC_PATH, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): f.write(chunk)

    print("[LOG] Processing Cards JSON...")
    with open(ATOMIC_PATH, "r", encoding="utf-8") as f: raw_data = json.load(f)
    all_cards = raw_data.get("data", {})

    seen_cards = set()
    processed_count = 0
    format_skipped = 0

    with open(CARDS_JSONL, "w", encoding="utf-8") as f_out:
        for card_name, faces in tqdm(all_cards.items(), unit="card"):
            if card_name in seen_cards: continue

            main_face = faces[0]
            legalities = main_face.get('legalities', {})

            # FORMAT CHECK: Must be legal in Paper Vintage or Commander
            if not (legalities.get('vintage') in ['Legal', 'Restricted'] or legalities.get('commander') in ['Legal', 'Restricted']):
                format_skipped += 1
                continue

            seen_cards.add(card_name)

            full_text = f"Card Name: {card_name}\n"
            for i, face in enumerate(faces):
                if len(faces) > 1: full_text += f"\n--- Face {i+1}: {face.get('name', card_name)} ---\n"
                if face.get('manaCost'): full_text += f"Cost: {clean_val(face.get('manaCost'))}\n"
                if face.get('type'): full_text += f"Type: {clean_val(face.get('type'))}\n"
                if 'power' in face: full_text += f"Stats: {face['power']}/{face['toughness']}\n"
                if face.get('text'): full_text += f"Oracle Text:\n{clean_val(face.get('text'))}\n"

            rulings = main_face.get('rulings', [])
            if rulings:
                full_text += "\nOfficial Rulings:\n"
                for r in rulings: full_text += f"- {r.get('text')}\n"

            f_out.write(json.dumps({"card_name": card_name, "text": full_text}, ensure_ascii=False) + "\n")
            processed_count += 1

    # --- STATISTICS ---
    print("\n" + "="*50)
    print("DATASET STATISTICS (CARDS)")
    print(f"   Cards Indexed:    {processed_count}")
    print(f"   Format Skipped:   {format_skipped} (Not Legal in Vintage/Commander)")
    print("="*50 + "\n")

# ==============================================================================
# PART B: INDEX FACTORY
# ==============================================================================

def get_indices(force_rebuild=False):
    Settings.embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL_NAME)
    Settings.llm = OpenAI(model=LLM_MODEL_NAME, temperature=0.0)

    # RULES
    if force_rebuild and RULES_STORAGE.exists(): shutil.rmtree(RULES_STORAGE)
    if not RULES_STORAGE.exists():
        print("[LOG] Building Rules Index...")
        RULES_STORAGE.mkdir(parents=True, exist_ok=True)
        if not CR_TXT_PATH.exists():
            resp = requests.get(CR_URL); CR_TXT_PATH.write_text(clean_unicode(resp.content.decode("utf-8-sig")), encoding="utf-8")
        rules_data = parse_rules_logic(CR_TXT_PATH)

        # SAMPLING
        rule_nodes = [r for r in rules_data if r['chapter_id'] != 'G']
        glossary_nodes = [r for r in rules_data if r['chapter_id'] == 'G']
        if rule_nodes: print(f"\n[SAMPLE RULE] {json.dumps(random.choice(rule_nodes), indent=2)}")
        if glossary_nodes: print(f"\n[SAMPLE GLOSSARY] {json.dumps(random.choice(glossary_nodes), indent=2)}")

        rules_docs = [Document(text=r["text"], metadata={k:v for k,v in r.items() if k!="text"}, excluded_embed_metadata_keys=["chapter_id","section_id","chapter_title","section_title"], excluded_llm_metadata_keys=["chapter_id","section_id"]) for r in rules_data]
        rules_index = VectorStoreIndex.from_documents(rules_docs, show_progress=True)
        rules_index.storage_context.persist(persist_dir=str(RULES_STORAGE))
    else:
        print("[LOG] Loading Rules Index...")
        rules_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=str(RULES_STORAGE)))

    # CARDS
    if force_rebuild and CARDS_STORAGE.exists(): shutil.rmtree(CARDS_STORAGE)
    if not CARDS_STORAGE.exists():
        print("[LOG] Building Cards Index...")
        CARDS_STORAGE.mkdir(parents=True, exist_ok=True)
        if not CARDS_JSONL.exists(): process_cards_data()

        cards_raw = []
        with open(CARDS_JSONL, "r", encoding="utf-8") as f:
            for line in f: cards_raw.append(json.loads(line))

        # SAMPLING
        if cards_raw: print(f"\n[SAMPLE CARD] {json.dumps(random.choice(cards_raw), indent=2, ensure_ascii=False)}\n")

        cards_docs = [Document(text=d["text"], metadata={"card_name": d["card_name"]}) for d in cards_raw]
        cards_index = VectorStoreIndex.from_documents(cards_docs, show_progress=True)
        cards_index.storage_context.persist(persist_dir=str(CARDS_STORAGE))
    else:
        print("[LOG] Loading Cards Index...")
        cards_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=str(CARDS_STORAGE)))

    return rules_index, cards_index

# ==============================================================================
# PART C: THE ENGINE (V5.8 - LAYER-FIRST ARCHITECTURE)
# ==============================================================================

class MagicJudgeEngine:
    def __init__(self, rules_idx, cards_idx):
        self.llm = Settings.llm
        self.rules_retriever = VectorIndexRetriever(index=rules_idx, similarity_top_k=8)
        self.cards_index = cards_idx

    def _extract_explicit_cards(self, query: str) -> List[str]:
        return re.findall(r"\[\[(.*?)\]\]", query)

    def query(self, user_question: str):
        # 1. Retrieval
        explicit_cards = self._extract_explicit_cards(user_question)
        c_nodes = []

        if explicit_cards:
            print(f"[LOG] Explicit Mode Active: {explicit_cards}")
            for card_name in explicit_cards:
                filters = MetadataFilters(filters=[MetadataFilter(key="card_name", value=card_name)])
                exact_retriever = self.cards_index.as_retriever(filters=filters, similarity_top_k=1)
                exact_nodes = exact_retriever.retrieve(card_name)

                if exact_nodes:
                    print(f"   >>> Found Exact Match: {card_name}")
                    for node in exact_nodes: node.score = 1.0
                    c_nodes.extend(exact_nodes)
                else:
                    print(f"   >>> Failed Exact Match for '{card_name}'. Falling back.")
                    fallback_retriever = VectorIndexRetriever(index=self.cards_index, similarity_top_k=3)
                    c_nodes.extend(fallback_retriever.retrieve(card_name))

        general_retriever = VectorIndexRetriever(index=self.cards_index, similarity_top_k=3)
        c_nodes.extend(general_retriever.retrieve(user_question))
        r_nodes = self.rules_retriever.retrieve(user_question)

        # 2. Context Building
        context_parts = ["--- RELEVANT CARD DATABASE ENTRIES ---"]
        seen_ids = set()
        unique_nodes = []
        for n in c_nodes:
            if n.node_id not in seen_ids:
                seen_ids.add(n.node_id)
                unique_nodes.append(n)
        for n in unique_nodes: context_parts.append(n.text)

        context_parts.append("\n--- COMPREHENSIVE RULES & GLOSSARY ---")
        for n in r_nodes:
            rid = n.metadata['rule_id']
            prefix = f"[Glossary: {rid}]" if n.metadata.get('chapter_id') == 'G' else f"[Rule {rid}]"
            context_parts.append(f"{prefix} {n.text}")

        full_context = "\n\n".join(context_parts)

        # 3. System Prompt
        system_msg = (
            "You are 'Judge's Familiar', a helpful Magic: The Gathering rules assistant. "
            "Your mandate is to provide strictly accurate rulings based *only* on the provided context and the logic below.\n\n"

            "*** RULES ENGINE PROTOCOLS: Apply the following RULES, PROTOCOLS and DOCTRINES in ORDER, but DO NOT mention them in your output. ***\n"
            "1. **THE GOLDEN RULE**: Specific card text overrides general game rules (Rule 101.1). "
            "Crucially, **'Can't' overrides 'Can' or 'Does'**.\n\n"

            "2. **THE LAYER SYSTEM PROTOCOL (Rule 613)**:\n"
            "   - If continuous effects interact, apply them in Layer order.\n"
            "   - **CRITICAL INTERACTION (Rule 305.7)**: If an effect turns a land into a Basic Land type (Layer 4), it loses abilities in its *printed* text box.\n"
            "   - **HOWEVER**: It DOES NOT lose abilities granted to it by other resolved effects or triggers (which apply in Layer 6).\n"
            "   - *Example*: A Land Saga transformed by [[Blood Moon]] loses printed chapters (Layer 4) but keeps abilities gained from resolved Chapter triggers (Layer 6).\n\n"

            "3. **THE SAGA SURVIVAL DOCTRINE (Rule 704.5s)**: \n"
            "   - A Saga is sacrificed ONLY if it meets two conditions: (A) Counters >= Max Chapters AND (B) It has 'one or more chapter abilities'.\n"
            "   - **Logic Check**: If the Layer System (Protocol 2) determines the Saga lost its *printed* chapter abilities (e.g. via [[Blood Moon]]), Condition (B) FAILS.\n"
            "   - **Result**: The Saga is **NOT** sacrificed. It remains on the battlefield.\n\n"

            "4. **THE 'GRENADE' DOCTRINE (Stack Independence)**: \n"
            "   - Once an ability is activated or triggered, it exists independently of its source.\n"
            "   - If the source is removed (exiled/destroyed) in response, the ability **STILL RESOLVES**.\n"
            "   - Use 'Last Known Information' (LKI) to determine values if the source is gone.\n\n"

            "5. **THE LITERAL TEXT DOCTRINE**:\n"
            "   - Read the provided Oracle Text literally. Do not infer keywords that are not written.\n"
            "   - (e.g., [[Solitude]] exiles a creature; it does NOT grant Shroud/Hexproof unless the text says so).\n\n"

            "*** OUTPUT INSTRUCTIONS ***\n"
            "1. **Tone**: Concise but Pedagogic. Explain the 'Why' briefly.\n"
            "2. **Layers**: Use Layer logic to determine the answer, but only mention specific Layers if relevant to the explanation.\n"
            "3. **Formatting**: Highlight cards: [[Card Name]].\n"
            "4. **Citations**: End with 'Sources: [Rule IDs / [[Card Names]]]'."
        )

        user_msg = f"Context:\n{full_context}\n\nQuestion: {user_question}\n\nExplanation:"

        stream_res = self.llm.stream_chat([
            ChatMessage(role=MessageRole.SYSTEM, content=system_msg),
            ChatMessage(role=MessageRole.USER, content=user_msg)
        ])

        sources = []
        for n in unique_nodes:
            sources.append({"id": n.metadata['card_name'], "type": "Card", "score": n.score})
        for n in r_nodes:
            sources.append({"id": n.metadata['rule_id'], "type": "Rule", "score": n.score})

        return stream_res, sources

# ==============================================================================
# PART D: RUNTIME
# ==============================================================================

rules_idx, cards_idx = get_indices(force_rebuild=FORCE_REBUILD)
judge = MagicJudgeEngine(rules_idx, cards_idx)

def ask_judge(question):
    print(f"\nQuestion: {question}\n")
    t0 = time.time()
    stream, sources = judge.query(question)

    print("Familiar: ", end="")
    try:
        response_gen = stream.response_gen if hasattr(stream, "response_gen") else stream
        for token in response_gen:
            content = token.delta if hasattr(token, "delta") else str(token)
            print(content, end="", flush=True)
    except Exception as e:
        print(f"\nError: {e}")

    print(f"\n\n{'='*60}")
    print(f"[LOG] RETRIEVAL CANDIDATES (Time: {time.time()-t0:.2f}s)")

    explicit = re.findall(r"\[\[(.*?)\]\]", question)
    if explicit:
        print(f"EXPLICIT MODE ACTIVE: {explicit}")
        print(f"{'-'*60}")

    sources.sort(key=lambda x: x['score'] if x['score'] else 0, reverse=True)
    seen = set()
    for s in sources:
        if s['type']=='Card' and s['id'] in seen: continue
        if s['type']=='Card': seen.add(s['id'])

        score_fmt = f"{s['score']:.2f}" if s['score'] else "1.00 (Exact)"
        print(f" - [{s['type']}] {s['id']:<30} (sc: {score_fmt})")
    print(f"{'='*60}\n")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 8.4 MB/s eta 0:00:00
[LOG] Building Rules Index...
[LOG] Starting Rules Parsing...
[LOG] Processing 2217 lines of glossary...

DATASET STATISTICS (RULES)
  

Parsing nodes:   0%|          | 0/3834 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1786 [00:00<?, ?it/s]

[LOG] Building Cards Index...
[LOG] Downloading AtomicCards.json...
[LOG] Processing Cards JSON...


...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1668 [00:00<?, ?it/s]

In [ ]:
ask_judge("If I attack with a creature with Deathtouch and Trample and it gets blocked, how much damage do I need to assign to the blocker?")


Question: If I attack with a creature with Deathtouch and Trample and it gets blocked, how much damage do I need to assign to the blocker?

Familiar: When you attack with a creature that has both Deathtouch and Trample and it gets blocked, you must assign at least 1 damage to the blocking creature. This is because any nonzero amount of combat damage assigned to a creature with Deathtouch is considered lethal damage. 

Since your creature has Trample, after assigning 1 damage to the blocker (which is lethal due to Deathtouch), you can assign any remaining damage to the defending player or planeswalker. For example, if your creature has 5 power, you would assign 1 damage to the blocker and the remaining 4 damage to the defending player.

This assignment is crucial because it allows you to maximize the damage dealt to the opponent while still fulfilling the requirement to assign lethal damage to the blocker.

Sources: [Rule 702.2c, Rule 702.19b, Rule 510.1]

[LOG] RETRIEVAL CANDIDATES (T

In [ ]:
ask_judge("I attack with [[Questing Beast]] and my opponent casts [[Fog]]. Does damage go through?")


Question: I attack with [[Questing Beast]] and my opponent casts [[Fog]]. Does damage go through?

[LOG] Explicit Mode Active: ['Questing Beast', 'Fog']
   >>> Found Exact Match: Questing Beast
   >>> Found Exact Match: Fog
Familiar: When you attack with [[Questing Beast]], it has the ability that states "Combat damage that would be dealt by creatures you control can't be prevented." Since [[Fog]] specifically prevents combat damage, it cannot prevent the damage dealt by [[Questing Beast]] due to this ability.

Therefore, the damage from [[Questing Beast]] will go through, and your opponent will take the combat damage as normal, despite the casting of [[Fog]].

Sources: [Rule 702.20b / [[Questing Beast]], [[Fog]]].

[LOG] RETRIEVAL CANDIDATES (Time: 19.34s)
EXPLICIT MODE ACTIVE: ['Questing Beast', 'Fog']
------------------------------------------------------------
 - [Card] Questing Beast                 (sc: 1.00)
 - [Card] Fog                            (sc: 1.00)
 - [Card] Heavy Fo

In [ ]:
ask_judge("If my opponent casts Solitude on my Ketramose during my turn, do I draw a card from Ketramose's ability?")


Question: If my opponent casts Solitude on my Ketramose during my turn, do I draw a card from Ketramose's ability?

Familiar: When your opponent casts [[Solitude]] targeting your [[Ketramose, the New Dawn]], Ketramose will be exiled, and its ability will trigger if one or more cards are put into exile from graveyards and/or the battlefield during your turn. 

However, since [[Solitude]] exiles Ketramose, it will not be on the battlefield to see the exile event. Therefore, Ketramose's ability will not trigger, and you will not draw a card. The ability requires Ketramose to be on the battlefield to trigger, and since it is exiled, it cannot fulfill that condition.

In summary, you do not draw a card from Ketramose's ability because it is not on the battlefield when the exile occurs.

Sources: [Rule 704.5s / [[Ketramose, the New Dawn]], [[Solitude]]].

[LOG] RETRIEVAL CANDIDATES (Time: 8.30s)
 - [Card] Ketramose, the New Dawn        (sc: 0.61)
 - [Card] Tempered in Solitude           (sc

In [ ]:
ask_judge("I cast [[Dress Down]] and my opponent has a [[Magus of the Moon]] in play. Are my non-basic lands still mountains?")


Question: I cast [[Dress Down]] and my opponent has a [[Magus of the Moon]] in play. Are my non-basic lands still mountains?

[LOG] Explicit Mode Active: ['Dress Down', 'Magus of the Moon']
   >>> Found Exact Match: Dress Down
   >>> Found Exact Match: Magus of the Moon
Familiar: Yes, your nonbasic lands are still Mountains. When you cast [[Dress Down]], it causes all creatures to lose their abilities, including the ability of [[Magus of the Moon]] that turns nonbasic lands into Mountains. However, the effect of [[Magus of the Moon]] that changes nonbasic lands into Mountains is applied before considering the loss of abilities from [[Dress Down]]. 

This means that while [[Magus of the Moon]] loses its abilities due to [[Dress Down]], the effect that has already turned your nonbasic lands into Mountains remains in effect. Therefore, your nonbasic lands will still function as Mountains, allowing you to tap them for {R}.

Sources: [Rule 305.7 / [[Dress Down]], [[Magus of the Moon]]].

[

In [ ]:
ask_judge("How can I play Obosh as my companion?")


Question: How can I play Obosh as my companion?

Familiar: To play [[Obosh, the Preypiercer]] as your companion, you must follow these steps:

1. **Deck Construction**: Ensure that your starting deck contains only cards with odd mana values and land cards. This is a requirement for Obosh's companion ability.

2. **Reveal Obosh**: At the beginning of the game, before shuffling your deck, you can reveal [[Obosh]] from outside the game (your sideboard) as your companion, provided your deck meets the requirements.

3. **Paying the Cost**: Once during the game, during your main phase when the stack is empty, you may pay {3} to put [[Obosh]] into your hand. This action is a special action and does not use the stack, meaning your opponents cannot respond to it.

4. **Casting Obosh**: After putting [[Obosh]] into your hand, you can cast it as you would any other card, provided you have the necessary mana available.

By following these steps, you can successfully utilize [[Obosh]] as your comp

In [ ]:
ask_judge("If I have an Urza's Saga on chapter II and my opponent plays a Blood Moon, what happens to my Urza's Saga?")


Question: If I have an Urza's Saga on chapter II and my opponent plays a Blood Moon, what happens to my Urza's Saga?

Familiar: When your opponent plays [[Blood Moon]], it turns your [[Urza's Saga]] into a basic land type (specifically, a Mountain). According to the rules, this transformation causes it to lose all of its printed chapter abilities, which are part of its text box. 

Since [[Urza's Saga]] has lost its chapter abilities, it no longer meets the condition of having "one or more chapter abilities" required for it to be sacrificed after it has accumulated enough lore counters. Therefore, even though it has two lore counters on it (indicating it is on chapter II), it will not be sacrificed because it no longer has any chapter abilities.

As a result, your [[Urza's Saga]] remains on the battlefield as a basic land, and you can still use it to generate mana, but it will not progress through its chapters or be sacrificed.

Sources: [Rule 613, Rule 704.5s / [[Urza's Saga]], [[Bloo

In [8]:
ask_judge("I cast a [[Dragonhawk, Fate's Tempest]] exiling 2 cards, but my opponent kills it right away with [[Shoot the Sheriff]]. On my end step, I still have the 2 cards exile. Does the Dragonhawk still do damage to my opponent even if it is in the graveyard?")


Question: I cast a [[Dragonhawk, Fate's Tempest]] exiling 2 cards, but my opponent kills it right away with [[Shoot the Sheriff]]. On my end step, I still have the 2 cards exile. Does the Dragonhawk still do damage to my opponent even if it is in the graveyard?

[LOG] Explicit Mode Active: ["Dragonhawk, Fate's Tempest", 'Shoot the Sheriff']
   >>> Found Exact Match: Dragonhawk, Fate's Tempest
   >>> Found Exact Match: Shoot the Sheriff
Familiar: Yes, the delayed triggered ability of [[Dragonhawk, Fate's Tempest]] will still deal damage to your opponent even if Dragonhawk is in the graveyard at that time. 

Here's why: When you cast Dragonhawk and it enters the battlefield, it exiles 2 cards from your library. This action creates a delayed triggered ability that will trigger at the beginning of your next end step, counting the cards exiled by that specific instance of Dragonhawk's ability. The important point is that the ability exists independently of Dragonhawk itself once it has bee

In [10]:
ask_judge("If my opponent controls [[The One Ring]] with 2 counters and activates it, and I exile it in response with [[Tear Asunder]], does my opponent draw 2 cards, 3 cards, or 0 cards?")


Question: If my opponent controls [[The One Ring]] with 2 counters and activates it, and I exile it in response with [[Tear Asunder]], does my opponent draw 2 cards, 3 cards, or 0 cards?

[LOG] Explicit Mode Active: ['The One Ring', 'Tear Asunder']
   >>> Found Exact Match: The One Ring
   >>> Found Exact Match: Tear Asunder
Familiar: When your opponent activates [[The One Ring]]'s ability, they put a burden counter on it and then attempt to draw a card for each burden counter on it. In this case, there are 2 burden counters, so they would normally draw 2 cards.

However, if you respond by casting [[Tear Asunder]] to exile [[The One Ring]], the ability still resolves because it exists independently of its source (the artifact). The ability will check the number of burden counters on [[The One Ring]] at the time it resolves, which is still 2, since the ability was activated before it was exiled.

Therefore, your opponent will draw 2 cards as a result of the ability resolving, despite [